In [1]:
# Import Data

import gdown
import pandas as pd

# Train Data
url = 'https://drive.google.com/uc?id=1-F7f9oSjFx0Si44DvdkzFhjjwfro_Y1p'
output = 'train_data.csv'
gdown.download(url, output, quiet=False)
train_data_df = pd.read_csv('train_data.csv')




Downloading...
From (uriginal): https://drive.google.com/uc?id=1-F7f9oSjFx0Si44DvdkzFhjjwfro_Y1p
From (redirected): https://drive.google.com/uc?id=1-F7f9oSjFx0Si44DvdkzFhjjwfro_Y1p&confirm=t&uuid=354d38f4-e446-431f-acb1-9ef51fd8fdd5
To: /jupyter/train_data.csv
100%|██████████| 635M/635M [00:28<00:00, 22.2MB/s] 
/tmp/ipykernel_274/1005448238.py:10: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  train_data_df = pd.read_csv('train_data.csv')


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import f1_score
import tensorflow as tf
from transformers import TFBertForSequenceClassification, BertTokenizer
import pandas as pd

# Assuming you have the dataset loaded in train_data_df
# Change the path or method according to your dataset loading process
# train_data_df = pd.read_csv('your_dataset.csv')
# تقسیم داده‌ها به دو بخش آموزش و آزمون
train_data, test_data = train_test_split(train_data_df, test_size=0.2, random_state=42)

# Calculate the number of samples to downsample to for each class
num_samples_per_class = 56756

# Resample the training data
resampled_train_data = pd.concat([
    resample(train_data[train_data['overall'] == 5], replace=True, n_samples=num_samples_per_class, random_state=42),
    resample(train_data[train_data['overall'] == 4], replace=True, n_samples=num_samples_per_class, random_state=42),
    resample(train_data[train_data['overall'] == 3], replace=True, n_samples=num_samples_per_class, random_state=42),
    resample(train_data[train_data['overall'] == 2], replace=True, n_samples=num_samples_per_class, random_state=42),
    resample(train_data[train_data['overall'] == 1], replace=True, n_samples=num_samples_per_class, random_state=42)
])
print(resampled_train_data['overall'].value_counts())

# استفاده از OneHotEncoder برای تبدیل ستون 'overall' به فرمت one-hot
encoder = OneHotEncoder(sparse=False)
y_train_onehot = encoder.fit_transform(resampled_train_data['overall'].values.reshape(-1, 1))
y_test_onehot = encoder.transform(test_data['overall'].values.reshape(-1, 1))

# Tokenize the input texts
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', max_length=128)
train_encodings = tokenizer(list(resampled_train_data["reviewText"]), truncation=True, padding=True)
test_encodings = tokenizer(list(test_data["reviewText"]), truncation=True, padding=True)

# Create TensorFlow datasets
batch_size = 8
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings), y_train_onehot)).shuffle(len(resampled_train_data)).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings), y_test_onehot)).batch(batch_size)

# Create the model
model = TFBertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=5)

# Compile the model with accuracy metric
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

# Train the model
model.fit(train_dataset, epochs=3)


# Evaluate the model for F1 score
predictions = model.predict(test_dataset)
predicted_labels = tf.argmax(predictions.logits, axis=1)
f1 = f1_score(test_data['overall'].values, predicted_labels, average='weighted')
print("F1 Score:", f1)

# Evaluate the model on the test dataset
results = model.evaluate(test_dataset)
accuracy = results[1]
print("Accuracy:", accuracy)


2023-11-12 03:07:38.354948: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-12 03:07:39.095770: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


overall
5    56756
4    56756
3    56756
2    56756
1    56756
Name: count, dtype: int64


/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
2023-11-12 03:25:25.324699: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22275 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:09:00.0, compute capability: 8.6
2023-11-12 03:39:29.172418: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You shoul

Epoch 1/3


2023-11-12 03:39:54.571635: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x1c4e83da0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-11-12 03:39:54.571676: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2023-11-12 03:39:54.579098: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-11-12 03:39:54.708978: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8700
2023-11-12 03:39:54.758708: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-11-12 03:39:54.822236: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


35473/35473 [==============================] - 8304s 233ms/step - loss: 0.9654 - accuracy: 0.5811
Epoch 2/3
35473/35473 [==============================] - 8207s 231ms/step - loss: 0.8240 - accuracy: 0.6468
Epoch 3/3
20974/20974 [==============================] - 1810s 86ms/step
F1 Score: 0.05806411918752488
20974/20974 [==============================] - 1849s 88ms/step - loss: 0.7972 - accuracy: 0.6711
Accuracy: 0.671122670173645


In [5]:
results = model.evaluate(test_dataset)
accuracy = results[1]
print("Accuracy:", accuracy)

# Transform one-hot encoded predictions back to labels
predicted_labels = tf.argmax(predictions.logits, axis=1)

# Transform one-hot encoded ground truth labels back to labels
true_labels = tf.argmax(y_test_onehot, axis=1)

# Evaluate the model for F1 score
f1 = f1_score(true_labels.numpy(), predicted_labels.numpy(), average='weighted')
print("F1 Score:", f1)


20974/20974 [==============================] - 1846s 88ms/step - loss: 0.7972 - accuracy: 0.6711
Accuracy: 0.671122670173645
F1 Score: 0.6911833956308225


In [6]:
# Test Data
url = 'https://drive.google.com/uc?id=1-8TsrqTRFP-q9TM-6HinhO0ZVXFHq9TB'
output = 'test_data.csv'
gdown.download(url, output, quiet=False)
test_data_df = pd.read_csv('test_data.csv')


Downloading...
From: https://drive.google.com/uc?id=1-8TsrqTRFP-q9TM-6HinhO0ZVXFHq9TB
To: /jupyter/test_data.csv
100%|██████████| 15.6M/15.6M [00:02<00:00, 6.27MB/s]


In [7]:
import pandas as pd
import tensorflow as tf
from transformers import BertTokenizer
from sklearn.metrics import f1_score


# Tokenize the input texts
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', max_length=128)
test_encodings = tokenizer(list(test_data_df["reviewText"]), truncation=True, padding=True)

# Create TensorFlow dataset for test data
batch_size = 8
test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings))).batch(batch_size)

# Make predictions on the test dataset
predictions = model.predict(test_dataset)
predicted_labels = tf.argmax(predictions.logits, axis=1)

# Map predicted labels back to original ratings (assuming labels are 0 to 4)
predicted_ratings = predicted_labels.numpy() + 1

# Add the 'overall' column to test_data_df
test_data_df['overall'] = predicted_ratings

# Save the updated DataFrame to a new CSV file
test_data_df.to_csv('predicted_test.csv', index=False)

# Note: Since we don't have ground truth labels for the test data, we can't evaluate F1 score or accuracy in this case.


2500/2500 [==============================] - 218s 86ms/step


In [8]:
import pandas as pd

# Load the predicted_test.csv file
predicted_test_df = pd.read_csv('predicted_test.csv')

# Extract the 'overall' column as 'predicted'
predicted_column = predicted_test_df['overall']

# Create a new DataFrame with the 'predicted' column
submission_df = pd.DataFrame({'predicted': predicted_column})

# Save the new DataFrame to q2_submission.csv without index
submission_df.to_csv('q2_submission.csv', index=False)
